## Import thư viện để crawl dữ liệu từ website <Ở đây là IEEE Xplore> (Dùng selenium, beautifulsoup lib của python)

In [57]:
from bs4 import BeautifulSoup

In [58]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver




Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:2 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [59]:
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

In [126]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

## Tạo chương trình để người dùng nhập tên tác giả mà muốn crawl dữ liệu từ trang IEEE Xplore

In [132]:
#Set up selenium in google collab
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  


In [133]:
#Ket noi voi trang chu cua website ieeexplore
driver.get("https://ieeexplore.ieee.org/Xplore/home.jsp")

In [134]:
#Tim thanh tim kiem cua website để nhập tên
search_bar = driver.find_element(By.CSS_SELECTOR, "input[aria-label='Enter search text']")

# Nguoi dung nhap ten cua tac gia can tim kiem
author = input("Nhap ten tac gia ma ban muon tim bai bao khoa hoc: ")

# Nhap ten ma nguoi dung vua nhap vao thanh tim kiem 
search_bar.clear()
search_bar.send_keys(author)
search_bar.send_keys(Keys.RETURN)

Nhap ten tac gia ma ban muon tim bai bao khoa hoc: Duy Dinh Le


In [135]:
#Luu anh tim kiem ten tac gia vao bo nho
driver.save_screenshot('WebsiteScreenShot.png')


True

## Tìm dữ liệu mong muốn để lấy, ở đây chúng ta sẽ tìm những dữ liệu như sau:
 1. Journal_title
 2. Authors
 3. Published_events
 4. Published_years 
 5. Conference_locations 
 6. Contents 

In [136]:
Journal_title = ""
Authors = ""
Published_events = ""
Published_years = ""
Conference_locations = ""
Contents = ""

In [139]:
number = 1
url = "https://ieeexplore.ieee.org/search/searchresult.jsp?newsearch=true&queryText=Duy%20Dinh%20Le"

with open(f"Scientific_journals_data.txt", "w") as file:
  for page_num in range(1, 4):
# Dung beautifulsoup de bat dau crawl data
    link = url + f'&highlight=true&returnFacets=ALL&returnType=SEARCH&matchPubs=true&pageNumber={page_num}'
    driver.get(link)
    time.sleep(10)
    soup_file=driver.page_source
    soup = BeautifulSoup(soup_file)
    results = soup.find_all('div', class_="col result-item-align")
    
    for result in results:
      detailed_link = 'https://ieeexplore.ieee.org' + str(result.h2.a['href']) + 'authors#authors'
      driver.get(detailed_link)
      time.sleep(10)
      soup = BeautifulSoup(driver.page_source)

      Journal_title = soup.find('h1', class_="document-title text-2xl-md-lh").text
      Contents = soup.find('div', class_="abstract-text row").text.replace("Abstract:", ' ').replace('.', '.\n')
      Published_events = soup.find('div', class_="u-pb-1 stats-document-abstract-publishedIn")
      Conference_locations = soup.find('div', class_="u-pb-1 doc-abstract-conferenceLoc")
      Published_years = soup.find('div', class_="u-pb-1 doc-abstract-confdate")
      for author in soup.find_all('div', class_="authors-accordion-container"):
        Authors = str(author.a.text)  + ' | ' + Authors 

      file.write(f"----  Scientific journal number {number}  ----")
      file.write(f"\nTitle: {Journal_title} ")
      file.write(f"\nAuthor: {Authors} ")
      if Published_years != None:
        file.write(f"\n{Published_years.text}")
      if Published_events != None:
        file.write(f"\n{Published_events.text}")
      if Conference_locations != None:
        file.write(f"\n{Conference_locations.text}")
      file.write(f"\nContent:\n{Contents}\n\n")
      number = number + 1


